In [ ]:
# this notebook shows how to obtain the basic results from Hunter et al. (2021)
# (corresponding to figs 3,4 & 5 / supp tables 2, 3, 6 & 8)
# it uses a newer version of the EM model fitting code with a different M step
# (& a newer version of julia etc)
# and substitutes Julia equivalents for Matlab analyses
#
# so it does not match results numerically but reproduces the key signif/nonsignif

In [1]:
# load some required packages
# ( you need to install these eg via import Pkg; Pkg.add(...) )

using CSV
using DataFrames
using LinearAlgebra
using SpecialFunctions:erf
using StatsFuns:logsumexp
using GLM
using Statistics
using StatsBase

In [2]:
# load the EM model fitting code 
# get this from https://github.com/ndawlab/em

directory = "/Users/ndaw/Dropbox/expts/julia em/git/em"

push!(LOAD_PATH,directory)
using EM

In [3]:
# patch in the python optimization function to produce results more similar to the original code
# (note you have to run this with only one thread / "unset JULIA_NUM_THREADS" ignoring warning)
# you can also skip this, & run julia multithreaded with the native optimizer to get (slightly
# different) results faster ;-)

using PyCall
using ForwardDiff

so = pyimport("scipy.optimize")

function EM.optimizesubject(likfun, startx)
    # call python's optimization function which used to work a little better than julia's
	# but is not thread safe
	a = so.minimize(likfun, startx, method="L-BFGS-B", jac = (x->ForwardDiff.gradient(likfun,x)))
	#println(a["message"])

	return((a["fun"],a["x"])::Tuple{Float64,Array{Float64,1}})
end

In [4]:
# load the model code (likelihood function)

include("patent likfuns.jl")

#3 (generic function with 1 method)

In [5]:
# load the data 

alldata = CSV.File("patent_investments_743subs.csv") |> DataFrame
exp1subs = unique(alldata.sub)[1:412]
exp1NS = length(exp1subs)

exp2subs = unique(alldata.sub)[413:end]
exp2NS = length(exp2subs)

allsubs = unique(alldata.sub)
allNS = length(allsubs)

743

In [6]:
# load the covariates
# make sure the covariates are z scored separately within cohorts

mergedcovars = CSV.File("mergedcovars.csv") |> DataFrame 
exp1covars = mergedcovars[1:412,:]
exp1covars.iqZ = zscore(exp1covars.iqZ)
exp1covars.lsasZ = zscore(exp1covars.lsasZ)

exp2covars = mergedcovars[413:end,:]
exp2covars.iqZ = zscore(exp2covars.iqZ)
exp2covars.lsasZ = zscore(exp2covars.lsasZ);

In [7]:
# Expt 1, basic model
# set up design matrix for fit

X = ones(exp1NS)

# start points

betas = [-1. 0 0 0 0 0 0 0 ]
sigma = [10.,1,1,1,1,1,1,1]

# fit the model

(betas,sigma,x,l,h) = em(alldata,exp1subs,X,betas,sigma,ewalik; emtol=1e-3, full=false, maxiter=1000);


iter: 667
betas: [1.23 1.2 -0.95 0.01 -0.34 -0.11 0.08 0.13]
sigma: [0.23, 0.61, 0.8, 0.01, 0.03, 0.02, 0.01, 0.01]
free energy: -31978.482233
change: [3.0e-6, 1.0e-6, -4.0e-6, 5.1e-5, -4.0e-6, -1.2e-5, 2.5e-5, 1.6e-5, 1.1e-5, 2.0e-6, 1.0e-6, 0.000999, 0.000685, 4.7e-5, 0.000547, 0.000658]
max: 0.000999


In [8]:
# x gives the per subject parameters (some need transforming to 0-1)
# extract them and regress on the covariates (originally we saved these and did regressions in Matlab)
# note significant effect of LSAS on w

exp1covars.beta = x[:,1]
exp1covars.alpha = 1 .- ( 0.5 .+ 0.5 .* erf.(x[:,2] ./ sqrt(2))) # alpha = 1-phi, squashed
exp1covars.w = ( 0.5 .+ 0.5 .* erf.(x[:,3] ./ sqrt(2)))

display(lm(@formula(beta~lsasZ+iqZ),exp1covars))
display(lm(@formula(alpha~lsasZ+iqZ),exp1covars))
display(lm(@formula(w~lsasZ+iqZ),exp1covars))

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, CholeskyPivoted{Float64, Matrix{Float64}}}}, Matrix{Float64}}

beta ~ 1 + lsasZ + iqZ

Coefficients:
──────────────────────────────────────────────────────────────────────────────
                  Coef.  Std. Error      t  Pr(>|t|)    Lower 95%    Upper 95%
──────────────────────────────────────────────────────────────────────────────
(Intercept)   1.23171      0.021374  57.63    <1e-99   1.18969      1.27372
lsasZ        -0.0456458    0.021404  -2.13    0.0336  -0.0877214   -0.00357026
iqZ           0.0366532    0.021404   1.71    0.0876  -0.00542236   0.0787288
──────────────────────────────────────────────────────────────────────────────

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, CholeskyPivoted{Float64, Matrix{Float64}}}}, Matrix{Float64}}

alpha ~ 1 + lsasZ + iqZ

Coefficients:
─────────────────────────────────────────────────────────────────────────────
                  Coef.  Std. Error      t  Pr(>|t|)    Lower 95%   Upper 95%
─────────────────────────────────────────────────────────────────────────────
(Intercept)   0.164033   0.00842711  19.46    <1e-59   0.147467    0.180599
lsasZ        -0.0078908  0.00843893  -0.94    0.3503  -0.0244799   0.00869829
iqZ           0.0110432  0.00843893   1.31    0.1914  -0.00554589  0.0276323
─────────────────────────────────────────────────────────────────────────────

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, CholeskyPivoted{Float64, Matrix{Float64}}}}, Matrix{Float64}}

w ~ 1 + lsasZ + iqZ

Coefficients:
──────────────────────────────────────────────────────────────────────────
                 Coef.  Std. Error      t  Pr(>|t|)   Lower 95%  Upper 95%
──────────────────────────────────────────────────────────────────────────
(Intercept)  0.217108   0.00952002  22.81    <1e-74  0.198394    0.235823
lsasZ        0.0224052  0.00953338   2.35    0.0192  0.00366464  0.0411457
iqZ          0.0224285  0.00953338   2.35    0.0191  0.00368799  0.041169
──────────────────────────────────────────────────────────────────────────

In [9]:
# Repeat the process with the valenced model, expt 1

# set up the design matrix

X = ones(exp1NS)

# start points 

betas = [-1. 0 0 0 0 0 0 0 0]
sigma = [10.,1,1,1,1,1,1,1,1]

# fit the model

(betas,sigma,x,l,h) = em(alldata,exp1subs,X,betas,sigma,ewalik_valenced; emtol=1e-3, full=false, maxiter=1000);


iter: 687
betas: [1.28 1.29 -0.64 -1.46 0.02 -0.34 -0.11 0.08 0.13]
sigma: [0.24, 0.65, 0.87, 1.28, 0.0, 0.03, 0.02, 0.01, 0.0]
free energy: -31736.91732
change: [2.0e-6, 1.0e-6, -2.0e-6, -7.0e-6, 6.2e-5, -5.0e-6, -1.5e-5, 2.9e-5, 1.8e-5, 1.1e-5, 1.1e-5, 5.0e-6, 1.8e-5, 0.000999, 0.000652, 0.000147, 0.000474, 0.000638]
max: 0.000999


In [10]:
# x gives the per subject parameters (some need transforming to 0-1)
# extract them and regress on the covariates (originally we saved these and did regressions in Matlab)
# note significant effect of LSAS on wplus, not wminus

exp1covars.betavalenced = x[:,1]
exp1covars.alphavalenced = 1 .- ( 0.5 .+ 0.5 .* erf.(x[:,2] ./ sqrt(2))) # alpha = 1-phi, squashed
exp1covars.wplus = ( 0.5 .+ 0.5 .* erf.(x[:,3] ./ sqrt(2)))
exp1covars.wminus = ( 0.5 .+ 0.5 .* erf.(x[:,4] ./ sqrt(2)))

display(lm(@formula(betavalenced~lsasZ+iqZ),exp1covars))
display(lm(@formula(alphavalenced~lsasZ+iqZ),exp1covars))
display(lm(@formula(wplus~lsasZ+iqZ),exp1covars)) # (the lsas effect comes up not quite signif in this version, python optimizer gives answer on other side of .05)
display(lm(@formula(wminus~lsasZ+iqZ),exp1covars))

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, CholeskyPivoted{Float64, Matrix{Float64}}}}, Matrix{Float64}}

betavalenced ~ 1 + lsasZ + iqZ

Coefficients:
─────────────────────────────────────────────────────────────────────────────
                  Coef.  Std. Error      t  Pr(>|t|)    Lower 95%   Upper 95%
─────────────────────────────────────────────────────────────────────────────
(Intercept)   1.27561     0.0214634  59.43    <1e-99   1.23342     1.3178
lsasZ        -0.0395631   0.0214935  -1.84    0.0664  -0.0818146   0.00268846
iqZ           0.0341602   0.0214935   1.59    0.1128  -0.00809136  0.0764117
─────────────────────────────────────────────────────────────────────────────

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, CholeskyPivoted{Float64, Matrix{Float64}}}}, Matrix{Float64}}

alphavalenced ~ 1 + lsasZ + iqZ

Coefficients:
──────────────────────────────────────────────────────────────────────────────
                   Coef.  Std. Error      t  Pr(>|t|)    Lower 95%   Upper 95%
──────────────────────────────────────────────────────────────────────────────
(Intercept)   0.150187    0.00827382  18.15    <1e-53   0.133923    0.166452
lsasZ        -0.00716422  0.00828543  -0.86    0.3877  -0.0234516   0.00912312
iqZ           0.0097024   0.00828543   1.17    0.2423  -0.00658494  0.0259897
──────────────────────────────────────────────────────────────────────────────

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, CholeskyPivoted{Float64, Matrix{Float64}}}}, Matrix{Float64}}

wplus ~ 1 + lsasZ + iqZ

Coefficients:
──────────────────────────────────────────────────────────────────────────
                 Coef.  Std. Error      t  Pr(>|t|)   Lower 95%  Upper 95%
──────────────────────────────────────────────────────────────────────────
(Intercept)  0.305959    0.011245   27.21    <1e-93  0.283854    0.328065
lsasZ        0.0268586   0.0112608   2.39    0.0175  0.00472233  0.048995
iqZ          0.0359935   0.0112608   3.20    0.0015  0.0138572   0.0581298
──────────────────────────────────────────────────────────────────────────

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, CholeskyPivoted{Float64, Matrix{Float64}}}}, Matrix{Float64}}

wminus ~ 1 + lsasZ + iqZ

Coefficients:
────────────────────────────────────────────────────────────────────────────
                   Coef.  Std. Error      t  Pr(>|t|)   Lower 95%  Upper 95%
────────────────────────────────────────────────────────────────────────────
(Intercept)   0.115325    0.00855022  13.49    <1e-33   0.0985174  0.132133
lsasZ         0.0060682   0.00856222   0.71    0.4789  -0.0107632  0.0228997
iqZ          -0.00128978  0.00856222  -0.15    0.8803  -0.0181212  0.0155417
────────────────────────────────────────────────────────────────────────────

In [11]:
# Expt 2, basic model
# set up design matrix for fit

X = ones(exp2NS)

# initial conditions

betas = [-1. 0 0 0 0 0 0 0 ]
sigma = [10.,1,1,1,1,1,1,1]

# fit the model

(betas,sigma,x,l,h) = em(alldata,exp2subs,X,betas,sigma,ewalik; emtol=1e-3, full=false, maxiter=1000);


iter: 910
betas: [1.13 1.24 -1.08 -0.05 -0.26 -0.17 0.09 0.15]
sigma: [0.43, 0.68, 1.26, 0.01, 0.01, 0.0, 0.02, 0.01]
free energy: -27648.934546
change: [1.0e-6, 0.0, -1.0e-6, -0.000121, -2.4e-5, -4.1e-5, 6.7e-5, 3.7e-5, 3.0e-6, 1.0e-6, 1.0e-6, 0.000174, 0.000999, 0.00096, 7.8e-5, 0.000398]
max: 0.000999


In [12]:
# x gives the per subject parameters (some need transforming to 0-1)
# extract them and regresss on the covariates (originally we saved these and did regressions in Matlab)
# note significant effect of LSAS on w

exp2covars.beta = x[:,1]
exp2covars.alpha = 1 .- ( 0.5 .+ 0.5 .* erf.(x[:,2] ./ sqrt(2))) # alpha = 1-phi, squashed
exp2covars.w = ( 0.5 .+ 0.5 .* erf.(x[:,3] ./ sqrt(2)))

display(lm(@formula(beta~lsasZ+iqZ),exp2covars))
display(lm(@formula(alpha~lsasZ+iqZ),exp2covars))
display(lm(@formula(w~lsasZ+iqZ),exp2covars))

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, CholeskyPivoted{Float64, Matrix{Float64}}}}, Matrix{Float64}}

beta ~ 1 + lsasZ + iqZ

Coefficients:
────────────────────────────────────────────────────────────────────────────
                   Coef.  Std. Error      t  Pr(>|t|)   Lower 95%  Upper 95%
────────────────────────────────────────────────────────────────────────────
(Intercept)   1.13069      0.0333753  33.88    <1e-99   1.06503    1.19634
lsasZ        -0.00173646   0.033437   -0.05    0.9586  -0.0675145  0.0640416
iqZ           0.115738     0.033437    3.46    0.0006   0.0499601  0.181516
────────────────────────────────────────────────────────────────────────────

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, CholeskyPivoted{Float64, Matrix{Float64}}}}, Matrix{Float64}}

alpha ~ 1 + lsasZ + iqZ

Coefficients:
───────────────────────────────────────────────────────────────────────────
                 Coef.  Std. Error      t  Pr(>|t|)    Lower 95%  Upper 95%
───────────────────────────────────────────────────────────────────────────
(Intercept)  0.161999   0.00943595  17.17    <1e-46   0.143436    0.180561
lsasZ        0.0156189  0.0094534    1.65    0.0995  -0.00297803  0.0342159
iqZ          0.010175   0.0094534    1.08    0.2826  -0.00842194  0.0287719
───────────────────────────────────────────────────────────────────────────

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, CholeskyPivoted{Float64, Matrix{Float64}}}}, Matrix{Float64}}

w ~ 1 + lsasZ + iqZ

Coefficients:
────────────────────────────────────────────────────────────────────────────
                  Coef.  Std. Error      t  Pr(>|t|)    Lower 95%  Upper 95%
────────────────────────────────────────────────────────────────────────────
(Intercept)  0.204007     0.0130319  15.65    <1e-40   0.17837     0.229643
lsasZ        0.033322     0.013056    2.55    0.0112   0.00763802  0.059006
iqZ          0.00364774   0.013056    0.28    0.7801  -0.0220363   0.0293317
────────────────────────────────────────────────────────────────────────────

In [13]:
# Repeat the process with the valenced model, expt 2

# set up the design matrix

X = ones(exp2NS)

betas = [-1. 0 0 0 0 0 0 0 0]
sigma = [10.,1,1,1,1,1,1,1,1]

(betas,sigma,x,l,h) = em(alldata,exp2subs,X,betas,sigma,ewalik_valenced; emtol=1e-3, full=false, maxiter=1000);


iter: 722
betas: [1.19 1.29 -0.87 -1.25 -0.04 -0.27 -0.17 0.11 0.16]
sigma: [0.34, 0.68, 1.11, 1.27, 0.01, 0.01, 0.01, 0.01, 0.01]
free energy: -27433.188284
change: [2.0e-6, 1.0e-6, -3.0e-6, -5.0e-6, -6.8e-5, -1.7e-5, -2.6e-5, 2.2e-5, 1.4e-5, 8.0e-6, 2.0e-6, 1.0e-6, 4.0e-6, 0.000545, 0.001, 0.000912, 0.000317, 0.000445]
max: 0.001


In [14]:
# x gives the per subject parameters (some need transforming to 0-1)
# extract them and regresss on the covariates (originally we saved these and did regressions in Matlab)
# note significant effect of LSAS on wplus not wminus

exp2covars.betavalenced = x[:,1]
exp2covars.alphavalenced = 1 .- ( 0.5 .+ 0.5 .* erf.(x[:,2] ./ sqrt(2))) # alpha = 1-phi, squashed
exp2covars.wplus = ( 0.5 .+ 0.5 .* erf.(x[:,3] ./ sqrt(2)))
exp2covars.wminus = ( 0.5 .+ 0.5 .* erf.(x[:,4] ./ sqrt(2)))

display(lm(@formula(betavalenced~lsasZ+iqZ),exp2covars))
display(lm(@formula(alphavalenced~lsasZ+iqZ),exp2covars))
display(lm(@formula(wplus~lsasZ+iqZ),exp2covars))
display(lm(@formula(wminus~lsasZ+iqZ),exp2covars))

# also regress on additional psychiatric symptom factors.
# f3 is the one measuring social anxiety (significant against wplus not wminus)
# f1 & f2 control for other symptom dimensions (not signif)

display(lm(@formula(betavalenced~f1+f2+f3+iqZ),exp2covars))
display(lm(@formula(alphavalenced~f1+f2+f3+iqZ),exp2covars))
display(lm(@formula(wplus~f1+f2+f3+iqZ),exp2covars))
display(lm(@formula(wminus~f1+f2+f3+iqZ),exp2covars))

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, CholeskyPivoted{Float64, Matrix{Float64}}}}, Matrix{Float64}}

betavalenced ~ 1 + lsasZ + iqZ

Coefficients:
───────────────────────────────────────────────────────────────────────────
                  Coef.  Std. Error      t  Pr(>|t|)   Lower 95%  Upper 95%
───────────────────────────────────────────────────────────────────────────
(Intercept)   1.19397     0.0293161  40.73    <1e-99   1.1363     1.25164
lsasZ        -0.0149823   0.0293703  -0.51    0.6103  -0.0727603  0.0427956
iqZ           0.117881    0.0293703   4.01    <1e-04   0.0601028  0.175659
───────────────────────────────────────────────────────────────────────────

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, CholeskyPivoted{Float64, Matrix{Float64}}}}, Matrix{Float64}}

alphavalenced ~ 1 + lsasZ + iqZ

Coefficients:
────────────────────────────────────────────────────────────────────────────
                  Coef.  Std. Error      t  Pr(>|t|)    Lower 95%  Upper 95%
────────────────────────────────────────────────────────────────────────────
(Intercept)  0.152579    0.0093286   16.36    <1e-43   0.134228    0.170931
lsasZ        0.0128441   0.00934585   1.37    0.1703  -0.00554124  0.0312295
iqZ          0.00776502  0.00934585   0.83    0.4067  -0.0106204   0.0261504
────────────────────────────────────────────────────────────────────────────

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, CholeskyPivoted{Float64, Matrix{Float64}}}}, Matrix{Float64}}

wplus ~ 1 + lsasZ + iqZ

Coefficients:
───────────────────────────────────────────────────────────────────────────
                  Coef.  Std. Error      t  Pr(>|t|)   Lower 95%  Upper 95%
───────────────────────────────────────────────────────────────────────────
(Intercept)  0.255575     0.0134084  19.06    <1e-54   0.229198   0.281953
lsasZ        0.0462293    0.0134332   3.44    0.0007   0.0198033  0.0726554
iqZ          0.00824102   0.0134332   0.61    0.5400  -0.018185   0.034667
───────────────────────────────────────────────────────────────────────────

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, CholeskyPivoted{Float64, Matrix{Float64}}}}, Matrix{Float64}}

wminus ~ 1 + lsasZ + iqZ

Coefficients:
─────────────────────────────────────────────────────────────────────────────
                   Coef.  Std. Error      t  Pr(>|t|)   Lower 95%   Upper 95%
─────────────────────────────────────────────────────────────────────────────
(Intercept)   0.156697     0.0112417  13.94    <1e-34   0.134582   0.178812
lsasZ         0.00740129   0.0112625   0.66    0.5115  -0.0147546  0.0295572
iqZ          -0.0180718    0.0112625  -1.60    0.1095  -0.0402277  0.00408417
─────────────────────────────────────────────────────────────────────────────

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, CholeskyPivoted{Float64, Matrix{Float64}}}}, Matrix{Float64}}

betavalenced ~ 1 + f1 + f2 + f3 + iqZ

Coefficients:
───────────────────────────────────────────────────────────────────────────
                  Coef.  Std. Error      t  Pr(>|t|)   Lower 95%  Upper 95%
───────────────────────────────────────────────────────────────────────────
(Intercept)   1.19397     0.0293431  40.69    <1e-99   1.13624    1.25169
f1           -0.0261705   0.0325766  -0.80    0.4224  -0.0902573  0.0379163
f2            0.020056    0.0298137   0.67    0.5016  -0.0385957  0.0787076
f3           -0.0160891   0.0327853  -0.49    0.6239  -0.0805866  0.0484083
iqZ           0.117773    0.0293975   4.01    <1e-04   0.0599404  0.175606
───────────────────────────────────────────────────────────────────────────

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, CholeskyPivoted{Float64, Matrix{Float64}}}}, Matrix{Float64}}

alphavalenced ~ 1 + f1 + f2 + f3 + iqZ

Coefficients:
────────────────────────────────────────────────────────────────────────────
                  Coef.  Std. Error      t  Pr(>|t|)    Lower 95%  Upper 95%
────────────────────────────────────────────────────────────────────────────
(Intercept)  0.152579    0.0093271   16.36    <1e-43   0.13423     0.170928
f1           0.0108461   0.0103549    1.05    0.2957  -0.00952472  0.0312169
f2           0.00168883  0.00947669   0.18    0.8587  -0.0169544   0.020332
f3           0.00762334  0.0104212    0.73    0.4650  -0.012878    0.0281247
iqZ          0.00808655  0.00934438   0.87    0.3875  -0.0102963   0.0264694
────────────────────────────────────────────────────────────────────────────

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, CholeskyPivoted{Float64, Matrix{Float64}}}}, Matrix{Float64}}

wplus ~ 1 + f1 + f2 + f3 + iqZ

Coefficients:
────────────────────────────────────────────────────────────────────────────
                   Coef.  Std. Error      t  Pr(>|t|)   Lower 95%  Upper 95%
────────────────────────────────────────────────────────────────────────────
(Intercept)   0.255575     0.0134327  19.03    <1e-54   0.22915    0.282001
f1           -0.00698569   0.0149129  -0.47    0.6398  -0.0363234  0.022352
f2           -0.00032429   0.0136482  -0.02    0.9811  -0.0271739  0.0265253
f3            0.0485016    0.0150085   3.23    0.0014   0.0189759  0.0780273
iqZ           0.00860207   0.0134576   0.64    0.5231  -0.0178726  0.0350768
────────────────────────────────────────────────────────────────────────────

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, CholeskyPivoted{Float64, Matrix{Float64}}}}, Matrix{Float64}}

wminus ~ 1 + f1 + f2 + f3 + iqZ

Coefficients:
─────────────────────────────────────────────────────────────────────────────
                    Coef.  Std. Error      t  Pr(>|t|)   Lower 95%  Upper 95%
─────────────────────────────────────────────────────────────────────────────
(Intercept)   0.156697      0.0112645  13.91    <1e-34   0.134537   0.178857
f1            0.000428833   0.0125058   0.03    0.9727  -0.0241733  0.025031
f2            0.00593032    0.0114451   0.52    0.6047  -0.0165853  0.028446
f3            0.00749713    0.0125859   0.60    0.5518  -0.0172627  0.0322569
iqZ          -0.0179247     0.0112853  -1.59    0.1132  -0.040126   0.0042766
─────────────────────────────────────────────────────────────────────────────